In [28]:
#aaaaa


!wget https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.yaml


import yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

print(config["telegram"]["bot_token"])

--2025-07-05 18:59:47--  https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273 [text/plain]
Saving to: ‘config.yaml.9’

config.yaml.9       100%[===================>]     273  --.-KB/s    in 0s      

2025-07-05 18:59:47 (18.0 MB/s) - ‘config.yaml.9’ saved [273/273]



ScannerError: mapping values are not allowed here
  in "config.yaml", line 216, column 75